In [ ]:
pip install ultralytics

: 

In [ ]:
import ultralytics
ultralytics.__version__

In [ ]:
pip install tracker

In [ ]:
pip install sort

In [ ]:
import cv2
import pandas as pd
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

In [ ]:
# Import the model
model = YOLO("model/yolov8s.pt")